In [1]:
%cd ..

/mnt/SSD_Data/active_projects/transformer_to_lstm


In [2]:
import dataclasses
from pathlib import Path
import warnings

import nlp
import torch
import numpy as np
import torch.nn.functional as F
from transformers import BertForSequenceClassification
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.model_selection import train_test_split

from pytorch_helper_bot import (
    BaseBot, MovingAverageStatsTrackerCallback,  CheckpointCallback,
    LearningRateSchedulerCallback, MultiStageScheduler, Top1Accuracy,
    LinearLR, Callback
)

from nobita.models import get_sequence_model


try:
    from apex import amp
    APEX_AVAILABLE = True
except ModuleNotFoundError:
    APEX_AVAILABLE = False
# APEX_AVAILABLE = False

In [3]:
CACHE_DIR = Path("cache/")
CACHE_DIR.mkdir(exist_ok=True)

Reference:

    * https://github.com/huggingface/nlp/blob/master/notebooks/Overview.ipynb

In [4]:
class SST2Dataset(torch.utils.data.Dataset):
    def __init__(self, entries_dict, temperature=1):
        super().__init__()
        self.entries_dict = entries_dict
        self.temperature = temperature
    
    def __len__(self):
        return len(self.entries_dict["label"])
    
    def __getitem__(self, idx):
        return (
            self.entries_dict["input_ids"][idx],
            self.entries_dict["attention_mask"][idx].sum(), #input_lengths
            {
                "label": self.entries_dict["label"][idx], 
                "logits": self.entries_dict["logits"][idx] / self.temperature
            }
        )

In [5]:
train_dict, valid_dict, test_dict = torch.load(str(CACHE_DIR / "distill-dicts.jbl"))

In [6]:
# Instantiate a PyTorch Dataloader around our dataset
train_loader = torch.utils.data.DataLoader(SST2Dataset(train_dict, temperature=2), batch_size=64, shuffle=True)
valid_loader = torch.utils.data.DataLoader(SST2Dataset(valid_dict, temperature=2), batch_size=64, drop_last=False)
test_loader = torch.utils.data.DataLoader(SST2Dataset(test_dict, temperature=2), batch_size=64, drop_last=False)

In [7]:
ALPHA = 0
DISTILL_OBJECTIVE = torch.nn.MSELoss()

def distill_loss(logits, targets):
    distill_part = DISTILL_OBJECTIVE(
        logits.reshape(-1), targets["logits"].reshape(-1)
    ) / 2
    classification_part = F.cross_entropy(
        logits, targets["label"]
    )
    return ALPHA * classification_part + (1-ALPHA) * distill_part

In [8]:
bert_model = BertForSequenceClassification.from_pretrained(str(CACHE_DIR / "sst2_bert_uncased")).cpu()

In [9]:
bert_model.bert.embeddings.word_embeddings.weight.shape

torch.Size([30522, 768])

In [10]:
# Note: apex does not support weight dropping
model = get_sequence_model(
    voc_size=bert_model.bert.embeddings.word_embeddings.weight.shape[0],
    emb_size=bert_model.bert.embeddings.word_embeddings.weight.shape[1],
    pad_idx = 0,
    dropoute = 0,
    rnn_hid = 768,
    rnn_layers = 3,
    bidir = True,
    dropouth = 0.25,
    dropouti = 0.25,
    wdrop = 0,
    unit_type = "lstm",
    fcn_layers = [512, 2],
    fcn_dropouts = [0.25, 0.25],
    use_attention = True
)

In [11]:
model

SequenceModel(
  (embeddings): BasicEmbeddings(
    (encoder): Embedding(30522, 768, padding_idx=0)
  )
  (encoder): RNNStack(
    (rnns): ModuleList(
      (0): LSTM(768, 384, bidirectional=True)
      (1): LSTM(768, 384, bidirectional=True)
      (2): LSTM(768, 384, bidirectional=True)
    )
    (dropouti): LockedDropout()
    (dropouths): ModuleList(
      (0): LockedDropout()
      (1): LockedDropout()
      (2): LockedDropout()
    )
  )
  (fcn): AttentionFCN(
    (attention): Attention(768, return attention=False)
    (layers): ModuleList(
      (0): LinearBlock(
        (lin): Linear(in_features=768, out_features=512, bias=True)
        (drop): Dropout(p=0.25, inplace=False)
        (bn): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): LinearBlock(
        (lin): Linear(in_features=512, out_features=2, bias=True)
        (drop): Dropout(p=0.25, inplace=False)
        (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, 

In [12]:
# Copy the embedding weights to the LSTM model
try:
    model.embeddings.encoder.emb.weight.data = bert_model.bert.embeddings.word_embeddings.weight.data
except:
    model.embeddings.encoder.weight.data = bert_model.bert.embeddings.word_embeddings.weight.data

In [13]:
# Freeze the embedding layer
for param in model.embeddings.encoder.parameters():
    param.requires_grad = False

In [14]:
model = model.cuda()

In [15]:
# Use only leaf tensors
parameters = [x for x in model.parameters() if x.is_leaf and x.requires_grad]

In [16]:
del bert_model

In [17]:
optimizer = torch.optim.Adam(parameters, lr=1e-3, betas=(0.8, 0.99))
# optimizer = torch.optim.RMSprop(parameters, lr=0.01)

In [18]:
if APEX_AVAILABLE:
    model, optimizer = amp.initialize(
        model, optimizer, opt_level="O1"
    )

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


In [19]:
class TransposeCallback(Callback):
    def on_batch_inputs(self, bot, input_tensors, targets):
        input_tensors = [input_tensors[0].transpose(1, 0), input_tensors[1]]
        return input_tensors, targets

In [20]:
class DistillTop1Accuracy(Top1Accuracy):
    def __call__(self, truth, pred):
        truth = truth["label"]
        return super().__call__(truth, pred)

In [21]:
total_steps = len(train_loader) * 10

checkpoints = CheckpointCallback(
    keep_n_checkpoints=1,
    checkpoint_dir=CACHE_DIR / "distill_model_cache/",
    monitor_metric="loss"
)
lr_durations = [
    int(total_steps*0.2),
    int(np.ceil(total_steps*0.8))
]
break_points = [0] + list(np.cumsum(lr_durations))[:-1]
callbacks = [
    MovingAverageStatsTrackerCallback(
        avg_window=len(train_loader) // 8,
        log_interval=len(train_loader) // 10
    ),
    LearningRateSchedulerCallback(
        MultiStageScheduler(
            [
                LinearLR(optimizer, 0.01, lr_durations[0]),
                CosineAnnealingLR(optimizer, lr_durations[1])
            ],
            start_at_epochs=break_points
        )
    ),
    checkpoints,
    TransposeCallback()
]
    
bot = BaseBot(
    log_dir = CACHE_DIR / "distill_logs",
    model=model, 
    train_loader=train_loader,
    valid_loader=valid_loader, 
    clip_grad=10.,
    optimizer=optimizer, echo=True,
    criterion=distill_loss,
    callbacks=callbacks,
    pbar=False, use_tensorboard=False,
    use_amp=APEX_AVAILABLE,
    metrics=(DistillTop1Accuracy(),)
)

[INFO][06/21/2020 22:33:35] SEED: 9293
[INFO][06/21/2020 22:33:35] # of parameters: 34,474,242
[INFO][06/21/2020 22:33:35] # of trainable parameters: 11,033,346


In [22]:
print(total_steps)

bot.train(
    total_steps=total_steps,
    checkpoint_interval=len(train_loader) // 2
)
bot.load_model(checkpoints.best_performers[0][1])
checkpoints.remove_checkpoints(keep=0)

[INFO][06/21/2020 22:33:35] Optimizer Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.8, 0.99)
    eps: 1e-08
    initial_lr: 0.001
    lr: 0.001
    weight_decay: 0
)
[INFO][06/21/2020 22:33:35] Batches per epoch: 1053
[INFO][06/21/2020 22:33:35] ====================Epoch 1====================


10530
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0


/data/miniconda3/envs/rapids/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:118: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
[INFO][06/21/2020 22:33:38] Step   105 | loss 2.36601565 | lr: 5.94e-05 | 0.031s per step
[INFO][06/21/2020 22:33:41] Step   210 | loss 2.03602501 | lr: 1.09e-04 | 0.030s per step
[INFO][06/21/2020 22:33:44] Step   315 | loss 1.73496945 | lr: 1.58e-04 | 0.029s per step
[INFO][06/21/2020 22:33:48] Step   420 | loss 1.43175183 | lr: 2.08e-04 | 0.030s per step
[INFO][06/21/2020 22:33:51] Step   525 | loss 1.19598607 | lr: 2.57e-04 | 0

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0


[INFO][06/21/2020 22:35:18] Step  3465 | loss 0.32094788 | lr: 9.38e-04 | 0.029s per step
[INFO][06/21/2020 22:35:21] Step  3570 | loss 0.30802362 | lr: 9.28e-04 | 0.029s per step
[INFO][06/21/2020 22:35:24] Step  3675 | loss 0.30738085 | lr: 9.18e-04 | 0.029s per step
[INFO][06/21/2020 22:35:25] Metrics at step 3682:
[INFO][06/21/2020 22:35:25] loss: 0.30135431
[INFO][06/21/2020 22:35:25] accuracy: 84.63%
[INFO][06/21/2020 22:35:28] Step  3780 | loss 0.31131974 | lr: 9.07e-04 | 0.032s per step
[INFO][06/21/2020 22:35:31] Step  3885 | loss 0.30219898 | lr: 8.95e-04 | 0.029s per step
[INFO][06/21/2020 22:35:34] Step  3990 | loss 0.28517096 | lr: 8.83e-04 | 0.029s per step
[INFO][06/21/2020 22:35:37] Step  4095 | loss 0.29933181 | lr: 8.70e-04 | 0.030s per step
[INFO][06/21/2020 22:35:40] Step  4200 | loss 0.28143905 | lr: 8.56e-04 | 0.032s per step
[INFO][06/21/2020 22:35:41] Metrics at step 4208:
[INFO][06/21/2020 22:35:41] loss: 0.27238926
[INFO][06/21/2020 22:35:41] accuracy: 87.39%


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 131072.0


[INFO][06/21/2020 22:38:13] Step  9450 | loss 0.13385803 | lr: 4.01e-05 | 0.029s per step
[INFO][06/21/2020 22:38:14] Metrics at step 9468:
[INFO][06/21/2020 22:38:14] loss: 0.23111891
[INFO][06/21/2020 22:38:14] accuracy: 88.07%
[INFO][06/21/2020 22:38:14] ====================Epoch 10====================
[INFO][06/21/2020 22:38:16] Step  9555 | loss 0.14210797 | lr: 3.28e-05 | 0.031s per step
[INFO][06/21/2020 22:38:19] Step  9660 | loss 0.13052901 | lr: 2.62e-05 | 0.029s per step
[INFO][06/21/2020 22:38:22] Step  9765 | loss 0.13239639 | lr: 2.03e-05 | 0.028s per step
[INFO][06/21/2020 22:38:25] Step  9870 | loss 0.12321634 | lr: 1.51e-05 | 0.028s per step
[INFO][06/21/2020 22:38:28] Step  9975 | loss 0.12520859 | lr: 1.07e-05 | 0.030s per step
[INFO][06/21/2020 22:38:29] Metrics at step 9994:
[INFO][06/21/2020 22:38:29] loss: 0.23559640
[INFO][06/21/2020 22:38:29] accuracy: 87.61%
[INFO][06/21/2020 22:38:32] Step 10080 | loss 0.12608127 | lr: 7.06e-06 | 0.030s per step
[INFO][06/21/

In [27]:
bot.eval(valid_loader)

{'loss': (0.23111891254372552, '0.23111891'),
 'accuracy': (-0.8807339449541285, '88.07%')}

In [28]:
bot.eval(test_loader)

{'loss': (0.26100756665435404, '0.26100757'),
 'accuracy': (-0.8738532110091743, '87.39%')}

In [26]:
# tokenizer.pad_token_id